In [46]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
import pandas as pd 
import numpy as np


sys.path.append(str(Path("../..").resolve()))

from src.constants import model_data_dir, raw_data_dir, raw_data_name
from src.utils import add_lagged_targets, add_seasonal_features

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
factors = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=1)
factors.set_index("Date", inplace=True)
categories = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=2)
outcomes = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=3)

In [88]:
sc_means = (factors.reset_index()
     .melt("Date", var_name = "Variable")
     .merge(categories, on="Variable")
     .groupby(["Date", "Subcategory"])
     .mean()
     .reset_index()
)
    

factors.reset_index().melt("Date", var_name = "Variable").merge(sc_means)

,Date,Variable,value,Subcategory
0,2000-12-26,V2X,NaN,Inflation
1,2000-12-26,V2X,NaN,Policy Uncertainty
2,2000-12-26,V2X,NaN,Sentiment
3,2000-12-26,V2X,NaN,VRP
4,2000-12-26,MSCI EM Vol,NaN,Inflation
...,...,...,...,...
418727,2006-11-22,EURUSD ATM Vol 1Y-3M,7.150,TERM/CARRY
418728,2010-09-02,USDJPY ATM Vol 1Y-3M,13.340,TERM/CARRY
418729,2010-09-02,EURUSD ATM Vol 1Y-3M,13.340,TERM/CARRY
418730,2020-07-02,USDJPY ATM Vol 1Y-3M,7.215,TERM/CARRY


In [99]:
sc_medians = factors.reset_index().melt("Date", var_name = "Variable", value_name="sc_value").merge(categories, on="Variable").groupby(["Date", "Subcategory"]).agg("mean").reset_index()


dd = factors.reset_index().melt("Date", var_name = "Variable").merge(sc_medians)    

In [37]:
lag = 7
lagged_targets = pd.DataFrame({f"lag_{i}": outcomes["value_1d_fwd_rel_d"].shift(i)[lag:] for i in range(1, lag+1)})
lagged_targets

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
7,0.0,0.0,0.0,1.0,0.0,1.0,0.0
8,1.0,0.0,0.0,0.0,1.0,0.0,1.0
9,1.0,1.0,0.0,0.0,0.0,1.0,0.0
10,0.0,1.0,1.0,0.0,0.0,0.0,1.0
11,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
5300,1.0,0.0,1.0,0.0,0.0,0.0,1.0
5301,1.0,1.0,0.0,1.0,0.0,0.0,0.0
5302,1.0,1.0,1.0,0.0,1.0,0.0,0.0
5303,0.0,1.0,1.0,1.0,0.0,1.0,0.0


In [49]:
outcomes

,Date,sc_1d_fwd_ret,mom_1d_fwd_ret,value_1d_fwd_ret,sp_1d_fwd_ret,sc_1d_fwd_rel_ret,mom_1d_fwd_rel_ret,value_1d_fwd_rel_ret,sc_1d_fwd_d,mom_1d_fwd_d,value_1d_fwd_d,sc_1d_fwd_rel_d,mom_1d_fwd_rel_d,value_1d_fwd_rel_d
0,2000-05-30,0.003626,-0.002746,-0.005446,-0.001301,0.004926,-0.001446,-0.004145,1,0,0,1,0,0
1,2000-05-31,0.022660,0.029970,0.024081,0.019858,0.002802,0.010112,0.004223,1,1,1,1,1,1
2,2000-06-01,0.052023,0.033394,0.016557,0.019637,0.032386,0.013757,-0.003080,1,1,1,1,1,0
3,2000-06-02,-0.003663,-0.001076,-0.001554,-0.006519,0.002856,0.005443,0.004965,0,0,0,1,1,1
4,2000-06-05,0.009804,-0.014203,-0.012033,-0.006671,0.016475,-0.007532,-0.005362,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5300,2021-06-23,0.014126,0.012622,0.008016,0.005811,0.008315,0.006811,0.002205,1,1,1,1,1,1
5301,2021-06-24,0.001682,0.002681,0.004598,0.003331,-0.001649,-0.000650,0.001267,1,1,1,0,0,1
5302,2021-06-25,-0.007921,0.001437,-0.001935,0.002315,-0.010237,-0.000878,-0.004250,0,1,0,0,0,0
5303,2021-06-28,-0.004774,0.000744,-0.003331,0.000277,-0.005051,0.000467,-0.003608,0,1,0,0,1,0
